<div style="display: table; width: 100%;">
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 70%;">
    <h1>Herramientas para Data Science</h1>
  </div>
  <div style="display: table-cell; text-align: center; vertical-align: middle; width: 30%;">
    <img src="https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/blob/main/Assets/UideLogo.png?raw=true" alt="logo UIDE" style="width:50%;">
  </div>
</div>
<hr />

### 🟦 Componente Práctico 1  
🟡 Grupo: 3      
🟡 Semana: 1      
🟡 Docente:  Ing. Iván García S., PhD. (idgs78@hotmail.com)     

### 🟦 Realizado por:   
Estudiantes

💻 Evelin Rosero Ordoñez   

💻 Marjorie Muso Tandalla

💻 José Espinoza Bone

### 🟦 Objetivo y alcance del trabajo 
- Esta práctica tiene el objetivo de realizar un preprocesamiento de datos y un Análisis 
exploratorio de datos (estadísticas y visualización) de las variables más relevantes 
que considere para el propósito de predicción de cáncer de mama. Al finalizar la 
práctica, los maestrantes podrán manipular adecuadamente un dataset de datos 
estructurados (alfanuméricos). [Breast Cancer Wisconsin (Diagnostic)](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)



- Esta práctica tiene el objetivo de realizar un preprocesamiento de datos y un Análisis 
exploratorio de datos (estadísticas y visualización) de las variables más relevantes 
que considere para el propósito de predicción de supervivencia en el desastre del Titanic. Al finalizar la 
práctica, los maestrantes podrán manipular adecuadamente un dataset de datos 
estructurados (alfanuméricos). [Titanic - Machine Learning from Disaster](https://www.kaggle.com/competitions/titanic)



### 🟦 Código fuente original
Con [git](https://git-scm.com/) instalado. En Windows, Linux o MacOS ejecutar el comando.

```
git clone "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1.git"
```






# 0️⃣ Preparar entorno

In [6]:
import sys
import subprocess
import os
from pathlib import Path
from enum import Enum
import zipfile
from typing import Optional, Iterable
from dataclasses import dataclass
from typing import cast
from typing import Tuple
from types import SimpleNamespace

# Libs a instalar
LIBS = [
    "numpy",
    "pandas",
    "matplotlib",
    "seaborn",
    "scikit-learn",
    "requests",
    "wcwidth",
]

class ConsoleColor(Enum):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    BLUE = "\033[94m"
    MAGENTA = "\033[95m"
    CYAN = "\033[96m"
    WHITE = "\033[97m"
    RESET = "\033[0m"


def PrintColor(message: str, color: ConsoleColor) -> str:
    RESET = ConsoleColor.RESET.value
    return f"{color.value}{message}{RESET}"


def ShowMessage(
    message: str, title: str, icon: str, color: ConsoleColor, end: str = "\n"
):
    colored_title = PrintColor(icon + f"  " + title.upper() + ":", color)
    print(f"{colored_title} {message}", end=end)


def ShowInfoMessage(
    message: str, title: str = "Info", icon: str = "ℹ️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.CYAN, end)


def ShowSuccessMessage(
    message: str, title: str = "Success", icon: str = "✅", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.GREEN, end)


def ShowErrorMessage(
    message: str, title: str = "Error", icon: str = "❌", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.RED, end)


def ShowWarningMessage(
    message: str, title: str = "Warning", icon: str = "⚠️", end: str = "\n"
):
    ShowMessage(message, title, icon, ConsoleColor.YELLOW, end)


# Funcion para ejecutar comandos
def RunCommand(
    commandList: list[str], printCommand: bool = True, printError: bool = True
) -> subprocess.CompletedProcess[str]:
    print("⏳", " ".join(commandList))

    if printCommand:
        proc = subprocess.Popen(
            commandList,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            bufsize=1,
            universal_newlines=True,
        )

        out_lines: list[str] = []
        assert proc.stdout is not None
        for line in proc.stdout:
            print(line, end="")
            out_lines.append(line)

        proc.wait()
        err_text = ""
        if proc.stderr is not None:
            err_text = proc.stderr.read() or ""

        if proc.returncode != 0 and printError and err_text:
            ShowErrorMessage(err_text, "", end="")
            # print(err_text, end="")

        return subprocess.CompletedProcess(
            args=commandList,
            returncode=proc.returncode,
            stdout="".join(out_lines),
            stderr=err_text,
        )

    else:
        result = subprocess.run(
            commandList, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        if result.returncode != 0 and printError and result.stderr:
            ShowErrorMessage(result.stderr, "", end="")
            # print(result.stderr, end="")
        return result


# Función para instalar las dependencias
def InstallDeps(libs: Optional[list[str]] = None):
    print("ℹ️ Installing deps.")
    printCommand = False
    printError = True
    RunCommand(
        [sys.executable, "-m", "pip", "install", "--upgrade", "pip"],
        printCommand=printCommand,
        printError=printError,
    )
    if libs is None or libs.count == 0:
        print("No hay elementos a instalar.")
    else:
        RunCommand(
            [sys.executable, "-m", "pip", "install", *libs],
            printCommand=printCommand,
            printError=printError,
        )
        print("Deps installed.")
    print()


# Función para mostrar info el ambiente de ejecución
def ShowEnvironmentInfo():
    print("ℹ️  Environment Info:")
    print("Python Version:", sys.version)
    print("Platform:", sys.platform)
    print("Executable Path:", sys.executable)
    print("Current Working Directory:", os.getcwd())
    print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))
    print("sys.prefix:", sys.prefix)
    print("sys.base_prefix:", sys.base_prefix)
    print()


InstallDeps(LIBS)
ShowEnvironmentInfo()
import requests


@dataclass(frozen=True)
class BoxStyle:
    TL: str
    TR: str
    BL: str
    BR: str
    H: str
    V: str

class TitleBoxLineStyle(Enum):
    SIMPLE = BoxStyle("┌", "┐", "└", "┘", "─", "│")
    DOUBLE = BoxStyle("╔", "╗", "╚", "╝", "═", "║")
    ROUNDED = BoxStyle("╭", "╮", "╰", "╯", "─", "│")
    HEAVY = BoxStyle("┏", "┓", "┗", "┛", "━", "┃")
    ASCII = BoxStyle("+", "+", "+", "+", "-", "|")
    DOUBLE_BOLD = BoxStyle("╔", "╗", "╚", "╝", "╬", "║")
    BLOCK = BoxStyle("█", "█", "█", "█", "█", "█")
    HEAVY_CROSS = BoxStyle("╒", "╕", "╘", "╛", "╪", "┃")
    METAL = BoxStyle("╞", "╡", "╘", "╛", "═", "║")


# Función para mostrar un título con recuadro
def ShowTitleBox(
    text: str,
    max_len: int = 100,
    boxLineStyle: TitleBoxLineStyle = TitleBoxLineStyle.SIMPLE,
    color: ConsoleColor = ConsoleColor.CYAN,
):
    try:

        def vislen(s: str) -> int:
            from wcwidth import wcswidth as _w

            n = _w(s)
            return n if n >= 0 else len(s)

    except Exception:

        def vislen(s: str) -> int:
            return len(s)

    pad = 1
    tlen = vislen(text)
    inner = max(max_len, tlen)
    left = (inner - tlen) // 2
    right = inner - tlen - left

    top = f"{boxLineStyle.value.TL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.TR}"
    mid = f"{boxLineStyle.value.V}{' ' * pad}{' ' * left}{text}{' ' * right}{' ' * pad}{boxLineStyle.value.V}"
    bot = f"{boxLineStyle.value.BL}{boxLineStyle.value.H * (inner + 2 * pad)}{boxLineStyle.value.BR}"
    print(PrintColor("\n".join([top, mid, bot]), color))


# Función para descargar un archivo
def DownloadFile(uri: str, filename: str, overwrite: bool = False, timeout: int = 20):
    dest = Path(filename).resolve()
    if dest.exists() and dest.is_file() and dest.stat().st_size > 0 and not overwrite:
        print(
            f'✅ Ya existe: "{dest}". No se descarga (use overwrite=True para forzar).'
        )
        return
    if dest.parent and not dest.parent.exists():
        dest.parent.mkdir(parents=True, exist_ok=True)
    print(f'ℹ️ Descargando "{uri}" → "{dest}"')
    try:
        with requests.get(uri, stream=True, timeout=timeout) as resp:
            resp.raise_for_status()
            tmp = dest.with_suffix(dest.suffix + ".part")
            with open(tmp, "wb") as f:
                for chunk in resp.iter_content(chunk_size=1024 * 64):
                    if chunk:  # filtra keep-alive chunks
                        f.write(chunk)
            tmp.replace(dest)
        print(f'✅ Archivo "{dest}" descargado exitosamente.')
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al descargar: {e}")


# Función para descomprimir un archivo zip
def UnzipFile(filename: str, outputDir: str):
    print(f'ℹ️ Descomprimiendo "{filename}" en "{outputDir}"')
    try:
        with zipfile.ZipFile(filename, "r") as zip_ref:
            zip_ref.extractall(outputDir)
        print(f"Descomprimido en: {os.path.abspath(outputDir)}")
    except Exception as e:
        print(f"Error: {e}")


ℹ️ Installing deps.
⏳ c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip
⏳ c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install numpy pandas matplotlib seaborn scikit-learn requests wcwidth
Deps installed.

ℹ️  Environment Info:
Python Version: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Platform: win32
Executable Path: c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe
Current Working Directory: c:\Users\Megam\OneDrive\Escritorio\6-Herramientas-Data-Science-Tarea1
VIRTUAL_ENV: None
sys.prefix: c:\Users\Megam\AppData\Local\Programs\Python\Python313
sys.base_prefix: c:\Users\Megam\AppData\Local\Programs\Python\Python313



In [7]:
# Importar libraries
import pandas as pd
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from matplotlib.figure import Figure
from matplotlib.axes import Axes

from dataclasses import dataclass
from typing import cast
from typing import Tuple
from enum import Enum

from pandas import DataFrame
from pandas import Series
from sklearn.utils import Bunch
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

# Configurar opciones de Pandas
pd.set_option("display.float_format", "{:.2f}".format)
pandas.set_option("display.max_rows", None)
pandas.set_option("display.max_columns", None) 

# Función para mostrar la información del DataFrame.
def ShowDfInfo(df: pandas.DataFrame, title):
    display(f"ℹ️ INFO {title} ℹ️")
    df.info()
    display()


# Función para mostrar las n primeras filas del DataFrame.
def ShowDfHead(df: pandas.DataFrame, title: str, headQty=10):
    display(f"ℹ️ {title}: Primeros {headQty} elementos.")
    display(df.head(headQty))
    display()


# Función para mostrar las n últimas filas del DataFrame.
def ShowDfTail(df: pandas.DataFrame, title: str, tailQty=10):
    display(f"ℹ️ {title}: Últimos {tailQty} elementos.")
    display(df.tail(tailQty))
    display()


# Mostrar el tamaño del DataFrame
def ShowDfShape(df: pandas.DataFrame, title: str):
    display(f"ℹ️ {title} - Tamaño de los datos")
    display(f"{df.shape[0]} filas x {df.shape[1]} columnas")
    display()


# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowDfStats(df: pandas.DataFrame, title: str = ""):
    display(f"ℹ️ Estadística descriptiva - {title}")
    numeric_cols = df.select_dtypes(include="number")
    if not numeric_cols.empty:
        display("    🔢 Columnas numéricas".upper())
        numeric_desc = (
            numeric_cols.describe().round(2).T
        )  # Transpuesta para añadir columna
        numeric_desc["var"] = numeric_cols.var(numeric_only=True).round(2)
        display(numeric_desc.T)
    non_numeric_cols = df.select_dtypes(
        include=["boolean", "string", "category", "object"]
    )
    if not non_numeric_cols.empty:
        display("    🔡 Columnas no numéricas".upper())
        non_numeric_desc = non_numeric_cols.describe()
        display(non_numeric_desc)
    datetime_cols = df.select_dtypes(include=["datetime", "datetimetz"])
    if not datetime_cols.empty:
        display("    📅 Columnas fechas".upper())
        datetime_desc = datetime_cols.describe()
        display(datetime_desc)

# Función para mostrar una visión general completa del DataFrame
def ShowFullDfOverview(df, title, headQty=5, tailQty=5):
    ShowDfInfo(df, title)
    ShowDfStats(df, title)
    ShowDfShape(df, title)
    ShowDfHead(df, title, headQty=headQty)
    ShowDfTail(df, title, tailQty=tailQty)

# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowDfNanValues(df: pandas.DataFrame, title: str):
    display(f"ℹ️ Contador de valores Nulos - {title}")
    nulls_count = df.isnull().sum()
    nulls_df = nulls_count.reset_index()
    nulls_df.columns = ["Columna", "Cantidad_Nulos"]
    display(nulls_df)
    display()


# Tipos de correlación
class CorrelationType(Enum):
    ALL = "all"
    STRONG = "strong"
    WEAK = "weak"

# Muestra las correlaciones completas, débiles y fuertes.
def ShowDfCorrelation(
    df: pandas.DataFrame,
    title: str,
    fig: Optional[Figure] = None,
    ax: Optional[Axes] = None,
    level: CorrelationType = CorrelationType.ALL,
    umbral: float = 0.6,
    showTable: bool = False,
    figsize: tuple = (8, 6),
    annotate: bool = True,
):
    if fig is None or ax is None:
        fig, ax = plt.subplots(figsize=figsize)

    display(f"ℹ️ {(title).upper()} - Matriz de Correlación, Type: {level.name}")

    corr = df.select_dtypes(include=["number"]).corr().copy()

    if level == CorrelationType.STRONG:
        corr = corr.where(np.abs(corr) >= umbral)
    elif level == CorrelationType.WEAK:
        corr = corr.where(np.abs(corr) < umbral)
        np.fill_diagonal(corr.values, 1)
    elif level != CorrelationType.ALL:
        raise ValueError(f"Invalid level: {level}")

    cax = ax.matshow(corr, vmin=-1, vmax=1)

    cols = corr.columns
    ax.set_xticks(range(len(cols)))
    ax.set_yticks(range(len(cols)))
    ax.set_xticklabels(cols, rotation=90, ha="left")
    ax.set_yticklabels(cols)

    fig.colorbar(cax)

    if annotate:
        for (i, j), value in np.ndenumerate(corr.values):
            if not np.isnan(value):
                ax.text(j, i, f"{value:+.2f}", ha="center", va="center")

    if level == CorrelationType.ALL:
        titulo = "Matriz de correlación completa"
    else:
        titulo = f"Matriz de correlación ({level.name}, umbral={umbral})"

    total_elementos = corr.size
    total_nodiagonal = corr.size - corr.shape[0]
    total_nan = corr.isna().sum().sum()
    total_validos = total_elementos - total_nan - corr.shape[0]

    titulo = (
        f"{titulo}, Total Matriz: {total_nodiagonal}, "
        f"Total válidos: {total_validos} ({((total_validos * 100) / total_nodiagonal):.2f}%)"
    )

    ax.set_title(titulo, pad=20)
    ax.grid(False)
    plt.tight_layout()
    plt.show()

    if showTable:
        display(corr)

    return fig, corr

def NormalizeColumnNames(df: pandas.DataFrame) -> pandas.DataFrame:
    df.columns = [
        col.strip().title().replace(" ", "").replace("_", "")
        for col in df.columns
    ]
    return df

def DropColumns(df: pandas.DataFrame, toDrop: list[str]):
    if not toDrop: 
        return df
    return df.drop(columns=[c.strip().title().replace(" ", "").replace("_", "") for c in toDrop if c in df.columns])

# Para almacenar los datos del dataset
@dataclass
class Dataset:
    X: pandas.DataFrame
    y: pandas.DataFrame

# Para almacenar los datos de split del dataset.
@dataclass
class DatasetSplit:
    Train: Dataset
    Test: Dataset

# Muestra el head de cada componente del split.
def ShowDatasetSplitHead(split: DatasetSplit, title: str, headQty: int = 5):
    ShowDfHead(split.Train.X, f"{title} - X Train", headQty)
    ShowDfHead(split.Train.y, f"{title} - y Train", headQty)
    ShowDfHead(split.Test.X, f"{title} - X Test", headQty)
    ShowDfHead(split.Test.y, f"{title} - y Test", headQty)

# Muestra la información del Dataset
def ShowDatasetInfo(data: Dataset, title):
    tAux = title
    title = f"{tAux} - Caracteristicas - X"
    ShowDfInfo(data.X, title)
    ShowDfShape(data.X, title)
    ShowDfStats(data.X, title)
    ShowDfNanValues(data.X, title)
    ShowDfHead(data.X, title)
    ShowDfTail(data.X, title)
    title = f"{tAux} - Características - y"
    ShowDfInfo(data.y, title)
    ShowDfShape(data.y, title)
    ShowDfStats(data.y, title)
    ShowDfNanValues(data.y, title)
    ShowDfHead(data.y, title)
    ShowDfTail(data.y, title)
    
# Muestra la información del Dataset Split
def ShowDatasetSplitInfo(split: DatasetSplit, title: str, headQty: int = 5):
    tAux = title
    title = f"{tAux} - TRAIN"
    ShowDatasetInfo(split.Train, title)
    title = f"{tAux} - TEST"
    ShowDatasetInfo(split.Test, title)

# Realiza el split del Dataset, en Train y test utilizando el ratio.
def SplitDataset(
    data: Dataset, trainRatio: float = 0.8, randomState: int = 42
) -> DatasetSplit:
    y_strat = data.y.iloc[:, 0]
    XTrain, XTest, yTrain, yTest = train_test_split(
        data.X,
        data.y,
        train_size=trainRatio,
        random_state=randomState,
        stratify=y_strat,
    )
    return DatasetSplit(
        Train=Dataset(X=XTrain.reset_index(drop=True), y=yTrain.reset_index(drop=True)),
        Test=Dataset(X=XTest.reset_index(drop=True), y=yTest.reset_index(drop=True)),
    )

# Para almacenar los datos del dataset aplicado el escalador.
@dataclass
class ScaledDatasetSplit(DatasetSplit):
    Scaler: StandardScaler

# Escala el split usando StandardScaler y retorna el split escalado.
def ScaleDatasetSplit(
    split: DatasetSplit, withMean: bool = True, withStd: bool = True
) -> ScaledDatasetSplit:
    scaler = StandardScaler(with_mean=withMean, with_std=withStd)
    XTrain = scaler.fit_transform(split.Train.X)
    XTest = scaler.transform(split.Test.X)
    XTrainScaled = split.Train.X.copy()
    XTestScaled = split.Test.X.copy()
    XTrainScaled.loc[:, :] = XTrain
    XTestScaled.loc[:, :] = XTest
    TrainScaled = Dataset(X=XTrainScaled, y=split.Train.y.copy())
    TestScaled = Dataset(X=XTestScaled, y=split.Test.y.copy())
    return ScaledDatasetSplit(Train=TrainScaled, Test=TestScaled, Scaler= scaler)





# TITANIC DATASET 1️⃣ Business Understanding - Entender el negocio

| Columna         | Descripción                                                                                                                                                  |
| --------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| **PassengerId** | Identificador único asignado a cada pasajero en el dataset. No tiene significado real en el viaje, solo sirve como índice.                                   |
| **Survived**    | Indica si el pasajero **sobrevivió (1)** o **murió (0)**. Es la variable objetivo en la mayoría de proyectos de Machine Learning sobre este dataset.         |
| **Pclass**      | Clase del boleto del pasajero: **1 = Primera clase**, **2 = Segunda clase**, **3 = Tercera clase**. Representa una medida indirecta de nivel socioeconómico. |
| **Name**        | Nombre completo del pasajero, generalmente incluye tratamiento o título social (Mr., Mrs., Miss, etc.).                                                      |
| **Sex**         | Sexo del pasajero: **male** (hombre) o **female** (mujer).                                                                                                   |
| **Age**         | Edad del pasajero en años. Puede contener valores faltantes. Los niños pequeños pueden aparecer con valores decimales.                                       |
| **SibSp**       | Número de **hermanos (siblings)** y **cónyuges (spouses)** que viajaban con el pasajero en el Titanic.                                                       |
| **Parch**       | Número de **padres (parents)** e **hijos (children)** que viajaban con el pasajero.                                                                          |
| **Ticket**      | Código del boleto del pasajero. Puede incluir letras y números; no cumple un formato estándar.                                                               |
| **Fare**        | Tarifa pagada por el pasajero por su boleto. El valor depende de la clase y otras condiciones del viaje.                                                     |
| **Cabin**       | Número de cabina asignada. Muchos valores están vacíos porque la mayoría de pasajeros no tenía cabina asignada (especialmente en 3ra clase).                 |
| **Embarked**    | Puerto de embarque: **C = Cherbourg**, **Q = Queenstown**, **S = Southampton**.                                                                              |


# TITANIC DATASET 2️⃣ Data preparation - Preparación de los datos
- Se descargan los archivos CSV. 
- Se eliminan columnas innecesarias.
- Se especifica bien los tipos de datos.
- Se aplica one hot encoding si es necesario a columnas nominales.

In [ ]:
DOWNLOADS_DIR = "Temp"
TITANIC_TEST_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicTest.csv"
TITANIC_TEST_GENDER_SUB_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicGenderSub.csv"
TITANIC_TRAIN_DATA_URL = "https://github.com/UIDE-Tareas/6-Herramientas-Data-Science-Tarea1/raw/refs/heads/main/Datasets/TitanicTrain.csv"
TITANIC_TEST_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicTest.csv")
TITANIC_TRAIN_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicTrain.csv")
TITANIC_TEST_GENDER_SUB_FILENAME = os.path.join(DOWNLOADS_DIR, "TitanicGenderSub.csv")


    
def LoadTitanicDatasetSplit(toDrop:list[str] = [])-> DatasetSplit:
    toDrop = [x for x in toDrop if x != "Survived"]
    DownloadFile(TITANIC_TEST_DATA_URL, TITANIC_TEST_FILENAME)
    DownloadFile(TITANIC_TRAIN_DATA_URL, TITANIC_TRAIN_FILENAME)
    DownloadFile(TITANIC_TEST_GENDER_SUB_DATA_URL, TITANIC_TEST_GENDER_SUB_FILENAME)
    dfTrain = pd.read_csv(TITANIC_TRAIN_FILENAME)
    dfTest = pd.read_csv(TITANIC_TEST_FILENAME)
    dfTestGenderSub = pd.read_csv(TITANIC_TEST_FILENAME)

    toDrop = ["Cabin", "PassengerId", "Ticket", "Name"]
    dfTrain = dfTrain.drop(columns=toDrop)
    dfTest  = dfTest.drop(columns=toDrop)

    dtypeMap = {
        "Survived": "int64",
        "Pclass": "int64",
        "Sex": "string",
        "Age": "float64",
        "SibSp": "int64",
        "Parch": "int64",
        "Fare": "float64",
        "Embarked": "string"
    }

    dfTrain = dfTrain.astype(dtypeMap)
    dfTest  = dfTest.astype({col: dtypeMap[col] for col in dfTest.columns})

    mapEmbarked = {
        "S": "Southampton",
        "C": "Cherbourg",
        "Q": "Queenstown"
    }
    dfTrain["Embarked"] = dfTrain["Embarked"].replace(mapEmbarked)
    dfTest["Embarked"]  = dfTest["Embarked"].replace(mapEmbarked)

    dfTrain = pd.get_dummies(dfTrain, columns=["Embarked"], prefix="Embarked")
    dfTest  = pd.get_dummies(dfTest, columns=["Embarked"], prefix="Embarked")

    dfTrain = pd.get_dummies(dfTrain, columns=["Sex"], prefix="Sex", drop_first=False)
    dfTest  = pd.get_dummies(dfTest, columns=["Sex"], prefix="Sex", drop_first=False)

    dfTrain["FamilySize"] = dfTrain["SibSp"] + dfTrain["Parch"] + 1
    dfTest["FamilySize"] = dfTest["SibSp"] + dfTest["Parch"] + 1

    meanAge = dfTrain["Age"].mean()
    dfTrain["Age"] = dfTrain["Age"].fillna(meanAge)
    dfTest["Age"]  = dfTest["Age"].fillna(meanAge)

    NormalizeColumnNames(dfTrain)
    NormalizeColumnNames(dfTest)

    dfTrain = DropColumns(dfTrain, toDrop)
    dfTest  = DropColumns(dfTest, toDrop)

    display(dfTrain.head(5))
    display(dfTest.head(5))
    print("-----------------------")
    XTrain = dfTrain.drop(columns=["Survived"])
    yTrain = dfTrain[["Survived"]]
    XTest  = dfTest.drop(columns=["Survived"])
    yTest  = dfTest[["Survived"]]
    return DatasetSplit(
        Train=Dataset(X=XTrain, y=yTrain),
        Test=Dataset(X=XTest, y=yTest)
    )



ShowTitleBox("EXPLORACIÓN INICIAL TITANIC DATASET", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.BLOCK)
ShowTitleBox("CARGANDO EL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
title = "Data original"
titanicDatasetSplit = LoadTitanicDatasetSplit()
ShowDatasetSplitInfo(titanicDatasetSplit, "Titanic Dataset Split", headQty=10)


CORR_UMBRAL = 0.3
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.ALL, showTable=True, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.STRONG, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.WEAK, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)






████████████████████████████████████████████████████████████████████████████████████████████████████████
█                                 EXPLORACIÓN INICIAL TITANIC DATASET                                  █
████████████████████████████████████████████████████████████████████████████████████████████████████████
┌──────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                         CARGANDO EL DATASET                                          │
└──────────────────────────────────────────────────────────────────────────────────────────────────────┘
✅ Ya existe: "C:\Users\Megam\OneDrive\Escritorio\6-Herramientas-Data-Science-Tarea1\Temp\TitanicTest.csv". No se descarga (use overwrite=True para forzar).
✅ Ya existe: "C:\Users\Megam\OneDrive\Escritorio\6-Herramientas-Data-Science-Tarea1\Temp\TitanicTrain.csv". No se descarga (use overwrite=True para forzar).


,Survived,Pclass,Age,Sibsp,Parch,Fare,EmbarkedCherbourg,EmbarkedQueenstown,EmbarkedSouthampton,SexFemale,SexMale,Familysize
0,0,3,22.00,1,0,7.25,False,False,True,False,True,2
1,1,1,38.00,1,0,71.28,True,False,False,True,False,2
2,1,3,26.00,0,0,7.92,False,False,True,True,False,1
3,1,1,35.00,1,0,53.10,False,False,True,True,False,2
4,0,3,35.00,0,0,8.05,False,False,True,False,True,1


-----------------------


KeyError: "['Survived'] not found in axis"

# BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET 1️⃣ Business Understanding - Entender el negocio

# Breast Cancer Wisconsin (Diagnostic) — Descripción de Columnas

El dataset contiene medidas computadas a partir de imágenes digitalizadas de tumores mamarios obtenidos mediante biopsia por aspiración con aguja fina (FNA).  
Incluye 569 observaciones y 32 columnas.

---

## 🧭 Columnas de Identificación y Diagnóstico

### **1. id**
Identificador único del registro. No se utiliza para entrenamiento.

### **2. diagnosis**
Diagnóstico final del tumor:
- **M** = Maligno  
- **B** = Benigno  

Es la **variable objetivo** del dataset.

---

## 🔬 Variables morfológicas del tumor

Cada variable aparece en tres versiones:

- **mean**: valor promedio  
- **se**: error estándar  
- **worst**: peor valor (percentil 90 o máximo observado)

---

## ⭐ Radius
Radio promedio del tumor, calculado como la distancia del centro al borde del núcleo.
- radius_mean  
- radius_se  
- radius_worst  

## ⭐ Texture
Desviación estándar de valores de intensidad de pixeles. Describe heterogeneidad.
- texture_mean  
- texture_se  
- texture_worst  

## ⭐ Perimeter
Perímetro del núcleo celular.
- perimeter_mean  
- perimeter_se  
- perimeter_worst  

## ⭐ Area
Área del núcleo celular, medida en píxeles.
- area_mean  
- area_se  
- area_worst  

## ⭐ Smoothness
Variación local del radio. Describe qué tan suave o irregular es la forma.
- smoothness_mean  
- smoothness_se  
- smoothness_worst  

## ⭐ Compactness
Calculado como:  
`(perimeter² / area) − 1.0`  
Indica cuán compacta es la forma.
- compactness_mean  
- compactness_se  
- compactness_worst  

## ⭐ Concavity
Grado de concavidad del borde del núcleo.
- concavity_mean  
- concavity_se  
- concavity_worst  

## ⭐ Concave Points
Número de puntos con concavidad en el borde.
- concave points_mean  
- concave points_se  
- concave points_worst  

## ⭐ Symmetry
Mide cuán simétrica es la forma de la célula.
- symmetry_mean  
- symmetry_se  
- symmetry_worst  

## ⭐ Fractal Dimension
Aproximación a la dimensión fractal del borde celular.
- fractal_dimension_mean  
- fractal_dimension_se  
- fractal_dimension_worst  

---


# BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET 2️⃣ Data preparation - Preparación de los datos

In [ ]:

# Carga el dataset
def LoadBreastCancerDataset() -> Dataset:
    bc = cast(Bunch, load_breast_cancer(as_frame=True))
    df: DataFrame = bc.frame.copy()
    TARGET_NAME = "target"
    X = df.drop(columns=[TARGET_NAME])
    NormalizeColumnNames(X)
    y = df[[TARGET_NAME]]
    y.columns = ["Diagnosis"]
    return Dataset(X, y)


ShowTitleBox("BREAST CANCER WISCONSIN (DIAGNOSTIC) DATASET", color=ConsoleColor.MAGENTA, boxLineStyle= TitleBoxLineStyle.BLOCK)
ShowTitleBox("CARGANDO EL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerDataset = LoadBreastCancerDataset()
title = "Data original"
ShowDatasetInfo(breastCancerDataset, title)

ShowTitleBox("HACIENDO SPLIT AL DATASET", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerSplit: DatasetSplit = SplitDataset(breastCancerDataset, trainRatio=0.8)
title = "Split del Dataset"
ShowDatasetSplitHead(breastCancerSplit, title)

ShowTitleBox("HACIENDO ESCALADO AL SPLIT", color=ConsoleColor.CYAN, boxLineStyle= TitleBoxLineStyle.SIMPLE)
breastCancerScaledSplit = ScaleDatasetSplit(breastCancerSplit)
title = "Split escalado"
ShowDatasetSplitHead(breastCancerScaledSplit, title)


CORR_UMBRAL = 0.3
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.ALL, showTable=True, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.STRONG, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)
ShowDfCorrelation(titanicDatasetSplit.Train.X, "Titanic TRAIN Data", level=CorrelationType.WEAK, showTable=False, figsize=(10,8), annotate=True, umbral=CORR_UMBRAL)